# Data Unfolding

Un compito importante dell’analisi dati in un esperimento consiste nel
dedurre, a partire dalla distribuzione misurata di una quantità fisica di interesse, la corrispondente distribuzione vera.

La differenza tra le due distribuzioni è dovuta sia alla presenza di un fondo fisico sia alle distorsioni causate dal rivelatore. 

Il metodo di correzione di queste distorsioni è noto con il nome di unfolding o deconvoluzione.

La diffrazione è un particolare fenomeno di interferenza che si verifica
quando un’onda incontra nel suo percorso un ostacolo o un’apertura. Il pattern di diffrazione cambia quando nel piano sono presenti delle fenditure multiple.

Lo spettro atteso è qualitativamente riportato in figura (c):






![](doppiaz.jpg)

Lo spettro di intensità può essere descritto come segue:

![](inter.png)


Si intende simulare lo spettro di intensità prodotto sullo schermo dalla
diffrazione da doppia fenditura.

In [1]:
import numpy as np
import ROOT
import pylab
import pandas
import random
import rootplot.root2matplotlib as r2m
from matplotlib import pyplot as plt
from ROOT import RooUnfoldResponse

# Definisco la funzione di I per 2 fenditure .
def doppia_fenditura (x , I_0 , a , L , lungh_onda) :
    return I_0 * np . power (( lungh_onda * L * 
                              np.sin ( np.pi * a * x /( lungh_onda * L ) ) 
                              /( np.pi * a * x )
                              *( np.sin ( np.pi * 2 * 0.001 * x /( lungh_onda * L 
                                                                 ) ) )
                              /( np.sin ( np.pi * 0.001 * x /( lungh_onda * L )
                                                                    ) ) ) , 2)

# Ridefinisco la funzione di I per N fenditure per ROOT .
def doppia_fenditura_root (x , parametri ) :
    return doppia_fenditura ( x[0] , parametri[0] , parametri[1] ,parametri [2] ,
                                                                     parametri[3])

# Costanti
e = 10
x_min = -0.02
x_max = +0.025
I_0 = 1.
L = 5.
a = 0.0005 
lungh_onda = 0.0000006
num_bins = 45
n = 50000

# Creo l oggetto TF1 e imposto i parametri .
funzione_doppia_fenditura = ROOT.TF1 ("funzione_doppia_fenditura",
                                        doppia_fenditura_root , x_min , x_max , 4)
funzione_doppia_fenditura.SetParameters (I_0 , a , L , lungh_onda)

# Creo l istogramma per i dati "veri".
histo = ROOT.TH1F("h", "Histo", num_bins , x_min , x_max)
c = ROOT.TCanvas("c")
    
# Creo lo spettro incidente generando n eventi e riempiendo l  istogramma.
x_i = []
for i in range( n ):
#    x_i = funzione_doppia_fenditura.GetRandom(x_min , x_max)
    x_i.append(funzione_doppia_fenditura.GetRandom(x_min , x_max))
    histo.Fill(x_i[i])
    
histo.Draw()
c.Draw()
c.SaveAs("doppia_fenditura_originale.png")


Welcome to JupyROOT 6.14/00


ModuleNotFoundError: No module named 'rootplot'

L'istogramma di eventi che si ottiene è:



![](doppia_fenditura_originale.png)

**Smearing dei dati**

Si applica ora uno smearing Gaussiano con  $\mu=0$ e $\sigma=c*\Delta x$ all'istogramma degli eventi ottenuti per introdurre la migrazione segnale tra bin come causa di distorsione.

In [ ]:
# Inizializzazioni .
mu_smear = 0.0
c = 1.0
Delta_x = (x_max - x_min)/num_bins

# Creo listogramma per i dati " misurati ".
d = ROOT.TCanvas("d")
histo_smeared = ROOT.TH1F("Smeared", "Smeared", num_bins , x_min , x_max)

# Applico lo smearing gaussiano ai dati generati in precedenza e 
# riempio un secondo istogramma .
for i in range(n):
    x_i_measured=x_i[i] + random.gauss(mu_smear , (c*Delta_x))
    histo_smeared.Fill(x_i_measured)
    
histo_smeared.Draw()
d.Draw()
d.SaveAs("histo_smeared.png")

Sono riportati tre differenti smearing rispettivamente con $c = 0.1$ ,  $c = 0.5$ e $c = 1.$ 

Si osserva una chiara “evoluzione” degli istogrammi con smearing all’aumentare di $c$; come atteso il picco centrale si abbassa e si allarga, mentre i picchi secondari sono sempre meno netti.

Smoothing con $c = 0.1$:

![](histo_smeared0.1.png)

Smoothing con $c = 0.5$:

![](histo_smeared0.5.png)

Smoothing con $c = 1$:

![](histo_smeared1.png)

**Unfolding dei dati**

Utilizzando il package RooUnfold si effettua quindi l'unfolding dei dati, ovvero “correggere” le distorsioni introdotte dal rivelatore e la
presenza di fondi.

Il metodo di unfolding che abbiamo utilizzato è il cosiddetto unfolding
bayesiano, che si basa sul noto teorema di Bayes e che venne introdotto nel 1994 da G. D’Agostini.

La procedura di unfolding bayesiano può essere spiegata in un quadro di
cause ed effetti nel quale le cause corrispondono ai valori veri prima dello smearing e gli effetti corrispondono ai valori misurati dopo lo smearing.

L’aspetto particolarmente interessante è la possibilità di effettuare l’unfolding anche in caso di completa ignoranza del processo di distorsione in esame; infatti è sufficiente porre $P_0(C)$ (la probabilità iniziale della causa $C$) uguale a una distribuzione uniforme. Scegliendo $P_0(C)$ uniforme sono necessarie più iterazioni rispetto al caso in cui la distribuzione iniziale è accuratamente scelta, ma non si corre il rischio di influenzare irreparabilmente la distribuzione finale.

Tipicamente dopo poche iterazioni la distribuzione finale approssima bene la distribuzione vera, mentre al crescere del numero di iterazioni di unfolding la distribuzione finale mostra grandi fluttuazioni attorno alla distribuzione vera. Questo effetto è dovuto a un’amplificazione delle fluttuazioni statistiche.

Si è scelta una distribuzione $P_0(C)$ uniforme e si è effettuato l’unfolding dei dati ottenuti si senza smearing che con smearing con i diversi valori di c. Ogni ciclo di unfolding è stato scelto essere composto da 4 iterazioni.

In [ ]:
# Inizializzione 
n_t = 1000000

e = ROOT.TCanvas("e")
# Creo un istogramma per lo spettro uniforme " vero " e
# un istogramma per lo spettro uniforme " misurato ".
histo_risp = ROOT.TH1F("Response_true","Response_true", num_bins ,
                                                       x_min , x_max)
histo_smeared_risp = ROOT.TH1F("Response_smeared" ,"Response_smeared" ,
                                               num_bins , x_min , x_max)


# Creo la matrice di risposta .
response = ROOT.RooUnfoldResponse(histo_smeared_risp ,histo_risp);
# Genero i due spettri e riempio la matrice di risposta .
for i in range (n_t):
    x_i_risp = random.uniform(x_min , x_max)
    x_i_measured_risp = random.gauss(x_i_risp , ( c * Delta_x ))
    response.Fill(x_i_measured_risp , x_i_risp)

plt.figure(1, (8, 6))
ax1 = plt.axes()
    
# Applico l'unfolding bayesiano ai dati ottenuti precedentemente 
# a seguito dello smearing gaussiano .
unfold_bayes = ROOT.RooUnfoldBayes(response , histo_smeared ,
                                           4 , False ) # Ciclo di 4 volte
Unfold_bayes = r2m.Hist(unfold_bayes.Hreco() , label = "Unfolded" ) 

Unfold_bayes.bar(color='r')
Unfold_bayes.show_titles()
plt.savefig('Unfolded_initial')
#plt.show()

# Recupero il valore di chi quadro .
#chi2 = unfold_bayes.Chi2

plt.clf()
plt.figure(1, (8, 6))
ax2 = plt.axes()

**Unfolding bayesiano con $c=1$**

![](Unfolded1.png)

**Unfolding bayesiano con $c=0.5$**

![](Unfolded05.png)

**Unfolding bayesiano con $c=0.1$**

![](Unfolded01.png)